# Plots for Training Dataset Size vs Accuracy

## Define Experiment Methods

### Training Dataset Size

In [ ]:
# Standard library
import json
import random
import sys

# My library
sys.path.append('../src/')
import util.mnist_loader
import networks.network as network
import networks.cnn_network as cnn_network

# Third-party libraries
import matplotlib.pyplot as plt
import numpy as np
from sklearn import svm
from keras.utils import to_categorical

# The sizes to use for the different training sets
SIZES = [100, 200, 500, 1000, 2000, 5000, 10000, 20000, 50000] 
                       
def run_networks():
    # Make results more easily reproducible
    random.seed(12345678)
    np.random.seed(12345678)
    training_data, validation_data, test_data = util.mnist_loader.load_data_wrapper()
    training_data = list(training_data)
    validation_data = list(validation_data)
    net = network.Network([784, 30, 10], cost=network.CrossEntropyCost())
    accuracies = []
    for size in SIZES:
        print("\n\nTraining network with data set size %s" % size)
        net.large_weight_initializer()
        num_epochs = round(1500000 / size)
        net.SGD(training_data[:size], num_epochs, 10, 0.5, lmbda = size*0.0001)
        accuracy = net.accuracy(validation_data) / 100.0
        print("Accuracy was %s percent" % accuracy)
        accuracies.append(accuracy)
    f = open("data/output/more_data.json", "w")
    json.dump(accuracies, f)
    f.close()

def run_svms():
    svm_training_data, svm_validation_data, svm_test_data \
        = util.mnist_loader.load_data()
    accuracies = []
    for size in SIZES:
        print("\n\nTraining SVM with data set size %s" % size)
        clf = svm.SVC()
        clf.fit(svm_training_data[0][:size], svm_training_data[1][:size])
        predictions = [int(a) for a in clf.predict(svm_validation_data[0])]
        accuracy = sum(int(a == y) for a, y in 
                       zip(predictions, svm_validation_data[1])) / 100.0
        print("Accuracy was %s percent" % accuracy)
        accuracies.append(accuracy)
    f = open("data/output/more_data_svm.json", "w")
    json.dump(accuracies, f)
    f.close()

def run_cnn():
    # Make results more easily reproducible
    random.seed(12345678)
    np.random.seed(12345678)
    # Get training data
    training_data_x, training_data_y, validation_data_x, validation_data_y = util.mnist_loader.load_data_cnn()
    # one hot encode target values
    training_data_y = to_categorical(training_data_y)
    validation_data_y = to_categorical(validation_data_y)
    accuracies = []
    for size in SIZES:
        print("\n\nTraining CNN with data set size %s" % size)
        cnn = cnn_network.CnnNetwork()
        num_epochs = round(500000 / size)
        # define model
        model = cnn.define_model()
        # Runn cnn training
        model.fit(training_data_x[:size], training_data_y[:size], epochs=num_epochs, batch_size=10, verbose=0)
        _, accuracy = model.evaluate(validation_data_x, validation_data_y)
        accuracy = accuracy*100
        print("Accuracy was %s percent" % accuracy)
        accuracies.append(accuracy)
    f = open("data/output/more_data_cnn.json", "w")
    json.dump(accuracies, f)
    f.close()

### Regularisation Parameter

In [ ]:
LMBDAS = [size*0.0001 for size in SIZES]

def run_regularisation_networks():
    # Make results more easily reproducible
    random.seed(12345678)
    np.random.seed(12345678)
    training_data, validation_data, test_data = util.mnist_loader.load_data_wrapper()
    training_data = list(training_data)
    validation_data = list(validation_data)
    net = network.Network([784, 30, 10], cost=network.CrossEntropyCost())
    accuracies = []
    for lmbda in LMBDAS:
        print(f"\n\nTraining network with lambda {lmbda}")
        net.large_weight_initializer()
        num_epochs = 30
        net.SGD(training_data, num_epochs, 10, 0.5, lmbda = lmbda)
        accuracy = net.accuracy(validation_data) / 100.0
        print("Accuracy was %s percent" % accuracy)
        accuracies.append(accuracy)
    f = open("data/output/more_lambda.json", "w")
    json.dump(accuracies, f)
    f.close()

## Gather Data

WARNING: Run at your own risk, will take a long time to train all networks

In [ ]:
# run_networks()
# run_svms()
# run_cnn()

In [ ]:
# run_regularisation_networks()

## Import Data

In [ ]:
# Standard library
import json

# Training Dataset Size
f = open("data/output/more_data.json", "r")
accuracies = json.load(f)
f.close()
f = open("data/output/more_data_svm.json", "r")
svm_accuracies = json.load(f)
f.close()
f = open("data/output/more_data_cnn.json", "r")
cnn_accuracies = json.load(f)
f.close()

# Regularisation Parameter
f = open("data/output/more_lambda.json", "r")
reg_accuracies = json.load(f)
f.close()

## 1. Make Combined Plot

In [ ]:
def make_combined_plot(accuracies, svm_accuracies, cnn_accuracies):
    fig = plt.figure(figsize=(20,10))
    ax = fig.add_subplot(111)
    ax.plot(SIZES, accuracies, color='#50BEDB')
    ax.plot(SIZES, accuracies, "o", color='#50BEDB', 
            label='Neural network accuracy (%)')
    ax.plot(SIZES, svm_accuracies, color='#3E91A8')
    ax.plot(SIZES, svm_accuracies, "o", color='#3E91A8',
            label='SVM accuracy (%)')
    ax.plot(SIZES, cnn_accuracies, color='#3E555C')
    ax.plot(SIZES, cnn_accuracies, "o", color='#3E555C',
            label='CNN accuracy (%)')
    ax.set_xlim(0, 50000)
    ax.set_ylim(60, 100)
    ax.set_xlabel('Training set size')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
make_combined_plot(accuracies, svm_accuracies, cnn_accuracies)

## 2. Make Hyperbolic Best Fit Plot

In [ ]:
def make_combined_bestfit_plot(accuracies, svm_accuracies, cnn_accuracies):
    fig = plt.figure(figsize=(20,10))
    ax = fig.add_subplot(111)
    ax.scatter(SIZES, accuracies, color='#50BEDB', label='Neural network accuracy (%)')
    # curve fit
    popt, _ = curve_fit(objective, SIZES, accuracies)
    # summarize the parameter values
    a, b, c, d = popt
    # define a sequence of inputs between the smallest and largest known inputs
    neural_x_line = range(1, 55000, 1)
    # calculate the output for the range
    neural_y_line = objective(neural_x_line, a, b, c, d)
    # create a line plot for the mapping function
    ax.plot(neural_x_line, neural_y_line, '--', color='#50BEDB')
    
    ax.scatter(SIZES, svm_accuracies, color='#3E91A8', label='SVM accuracy (%)')
    # curve fit
    popt, _ = curve_fit(objective, SIZES, svm_accuracies)
    # summarize the parameter values
    a, b, c, d = popt
    # define a sequence of inputs between the smallest and largest known inputs
    svm_x_line = range(0, 55000, 1)
    # calculate the output for the range
    svm_y_line = objective(svm_x_line, a, b, c, d)
    # create a line plot for the mapping function
    ax.plot(svm_x_line, svm_y_line, '--', color='#3E91A8')   
    
    ax.scatter(SIZES, cnn_accuracies, color='#3E555C', label='CNN accuracy (%)')
    # curve fit
    popt, _ = curve_fit(objective, SIZES, cnn_accuracies)
    # summarize the parameter values
    a, b, c, d = popt
    # define a sequence of inputs between the smallest and largest known inputs
    cnn_x_line = range(0, 55000, 1)
    # calculate the output for the range
    cnn_y_line = objective(cnn_x_line, a, b, c, d)
    # create a line plot for the mapping function
    ax.plot(cnn_x_line, cnn_y_line, '--', color='#3E555C')  
    
    fontProperties = {'fontname':'Arial', 'fontsize':'x-large'}
    
    ax.set_xlim(0, 55000)
    ax.set_ylim(60, 100)
    ax.set_xlabel('Training set size', **fontProperties)
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
from scipy.optimize import curve_fit
import numpy as np

# objective function
def objective(x, a, b, c, d):
	return a/(b*x-d) + c 

In [ ]:
make_combined_bestfit_plot(accuracies, svm_accuracies, cnn_accuracies)

## 3. Make Power Law Best Fit Plot

In [ ]:
def make_refined_combined_bestfit_plot(accuracies, svm_accuracies, cnn_accuracies, param_bounds):
    fig = plt.figure(figsize=(20,10))
    ax = fig.add_subplot(111)
    ax.scatter(SIZES, accuracies, color='#50BEDB', label='Neural network accuracy (%)')
    # curve fit
    popt, _ = curve_fit(objective, SIZES, accuracies, p0=[100,-100,-1], bounds=param_bounds)
    # summarize the parameter values
    a, c1, c2 = popt
    # define a sequence of inputs between the smallest and largest known inputs
    neural_x_line = range(1, 55000, 1)
    # calculate the output for the range
    neural_y_line = objective(neural_x_line, a, c1, c2)
    # create a line plot for the mapping function
    ax.plot(neural_x_line, neural_y_line, '--', color='#50BEDB')
    print(f"ANN Parameters are {popt} with SD: {np.sqrt(np.diag(_))}")
    
    ax.scatter(SIZES, svm_accuracies, color='#3E91A8', label='SVM accuracy (%)')
    # curve fit
    popt, _ = curve_fit(objective, SIZES, svm_accuracies, p0=[100,-100,-1], bounds=param_bounds)
    # summarize the parameter values
    a, c1, c2 = popt
    # define a sequence of inputs between the smallest and largest known inputs
    svm_x_line = range(0, 55000, 1)
    # calculate the output for the range
    svm_y_line = objective(svm_x_line, a, c1, c2)
    # create a line plot for the mapping function
    ax.plot(svm_x_line, svm_y_line, '--', color='#3E91A8')
    print(f"SVM Parameters are {popt} with SD: {np.sqrt(np.diag(_))}")
    
    ax.scatter(SIZES, cnn_accuracies, color='#3E555C', label='CNN accuracy (%)')
    # curve fit
    popt, _ = curve_fit(objective, SIZES, cnn_accuracies, p0=[100,-100,-1], bounds=param_bounds)
    # summarize the parameter values
    a, c1, c2 = popt
    # define a sequence of inputs between the smallest and largest known inputs
    cnn_x_line = range(0, 55000, 1)
    # calculate the output for the range
    cnn_y_line = objective(cnn_x_line, a, c1, c2)
    # create a line plot for the mapping function
    ax.plot(cnn_x_line, cnn_y_line, '--', color='#3E555C')
    print(f"CNN Parameters are {popt} with SD: {np.sqrt(np.diag(_))}")
    
    fontProperties = {'fontname':'Arial', 'fontsize':'x-large'}
    
    ax.set_xlim(0, 55000)
    ax.set_ylim(60, 100)
    ax.set_xlabel('Training set size', **fontProperties)
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
from scipy.optimize import curve_fit
import numpy as np

# objective function
def objective(x, a, c1, c2):
    return a + x**c2 * c1

param_bounds=([0,-np.inf,-np.inf],[100,0,0])

In [ ]:
make_refined_combined_bestfit_plot(accuracies, svm_accuracies, cnn_accuracies, param_bounds)

## 4. Plot Behaviour of Another Regulisation Method

In [ ]:
from scipy.optimize import curve_fit
import numpy as np

# objective function
def objective(x, a, c1, c2):
    return a + x**c2 * c1

param_bounds=([0,-np.inf,-np.inf],[100,0,0])

In [ ]:
def make_regularisation_plot(reg_accuracies, param_bounds):
    fig = plt.figure(figsize=(20,10))
    ax = fig.add_subplot(111)
    ax.scatter(LMBDAS, reg_accuracies, color='#50BEDB')
    
    popt, _ = curve_fit(objective, LMBDAS, reg_accuracies, bounds=param_bounds)
    # summarize the parameter values
    a, c1, c2 = popt
    # define a sequence of inputs between the smallest and largest known inputs
    neural_x_line = [ x/1000 for x in range(1, 6000, 1) ]
    # calculate the output for the range
    neural_y_line = objective(neural_x_line, a, c1, c2)
    # create a line plot for the mapping function
    ax.plot(neural_x_line, neural_y_line, '--', color='#50BEDB')
    print(f"Parameters are {popt} with SD: {np.sqrt(np.diag(_))}")
    
    fontProperties = {'fontname':'Arial', 'fontsize':'x-large'}
    
    ax.set_xlim(0, 7)
    ax.set_ylim(94, 97)
    ax.set_xlabel('Lambda', **fontProperties)
    plt.show()

In [ ]:
make_regularisation_plot(reg_accuracies, param_bounds)